In [14]:
# %preprocess physiological data files%
# compute values of zyg, cog, scr for each trigger in each trial
# combine physiological values to behavioral data file (one row per trial)
# coding=utf-8

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from scipy.stats import zscore
from constant import *
from pyphysio.pyphysio import signal, filters, segmenters, interactive
from pyphysio.pyphysio.specialized.eda import DriverEstim, PhasicEstim, preset_phasic
from pyphysio.pyphysio.specialized.emg._presets import preset_femg

In [15]:
# load data
def load(datafile, triggers):
    col_names = ["zyg","scr","cor","trg1","trg2","trg3","trg4","trg5","trg6","trg7","trg8","na"]
    data = pd.read_csv(datafile, sep='\t', names = col_names, index_col=False)
    data['line'] = np.array(list(np.linspace(0, data.shape[0]-1, data.shape[0])), int)
    data["binary"] = data.trg1.astype(str) + data.trg2.astype(str) + data.trg3.astype(str) + \
    data.trg4.astype(str) + data.trg5.astype(str) + data.trg6.astype(str) + \
    data.trg7.astype(str) + data.trg8.astype(str)
    df = data.merge(triggers, on="binary").drop(["Binary"], axis=1).sort_values(by='line')
    return df

In [16]:
# separate phases
def phases(df):
    df['Phase'] = ''
    line_idxs = {}
    phase_names = list(triggers['TriggerName'][triggers['TriggerName'].str.endswith(("Start", "End"))])
    if not ("expStart" in df['TriggerName']):
        df.loc[0,'TriggerName'] = "expStart"
    for phase in phase_names:
        # print(phase)
        # print(phase in set(df['TriggerName']))
        line_i = np.where(df['TriggerName']== phase)[0][0]
        line_n = df.line.iloc[line_i]
        line_idxs[phase]=line_n
    # Practice Start
    df.loc[df.line.isin(range(line_idxs['prac2BackStart'],line_idxs['caliStart'])),'Phase'] = "Practice2Back"
    df.loc[df.line.isin(range(line_idxs['caliStart'],line_idxs['prac1BackStart'])),'Phase'] = "Calibration"
    df.loc[df.line.isin(range(line_idxs['prac1BackStart'],line_idxs['prac3BackStart'])),'Phase'] = "Practice1Back"
    df.loc[df.line.isin(range(line_idxs['prac3BackStart'],line_idxs['testStart'])),'Phase'] = "Practice3Back"
    # Test Start
    df.loc[df.line.isin(range(line_idxs['offlineT0Start'],line_idxs['learningStart'])),'Phase'] = "OfflineRatingT0"
    df.loc[df.line.isin(range(line_idxs['learningStart'],line_idxs['offlineT1Start'])),'Phase'] = "Learning"
    df.loc[df.line.isin(range(line_idxs['offlineT1Start'],line_idxs['trueChoiceStart'])),'Phase'] = "OfflineRatingT1"
    df.loc[df.line.isin(range(line_idxs['trueChoiceStart'],line_idxs['offlineT2Start'])),'Phase'] = "TrueChoice"
    
    df.loc[df.line.isin(range(line_idxs['offlineT2Start'],line_idxs['learningCheckT1Start'])),'Phase'] = "OfflineRatingT2"
    df.loc[df.line.isin(range(line_idxs['learningCheckT1Start'],line_idxs['falseChoiceStart'])),'Phase'] = "LearningCheckT1"
    df.loc[df.line.isin(range(line_idxs['falseChoiceStart'],line_idxs['offlineT3Start'])),'Phase'] = "FalseChoice"
    df.loc[df.line.isin(range(line_idxs['offlineT3Start'],line_idxs['learningCheckT2Start'])),'Phase'] = "OfflineRatingT3"
    df.loc[df.line.isin(range(line_idxs['learningCheckT2Start'],line_idxs['demandRatingStart'])),'Phase'] = "LearningCheckT2"
    # Demand Manipulation Check
    df.loc[df.line.isin(range(line_idxs['demandRatingStart'],line_idxs['expEnd'])),'Phase'] = "DemandRating"
    # separate phases
    df_learning = df[df['Phase'] == "Learning"].copy().sort_values('line')
    df_true_choice = df[df['Phase'] == "TrueChoice"].copy().sort_values('line')
    df_false_choice = df[df['Phase'] == "FalseChoice"].copy().sort_values('line')
    # df_offlines = df[df['Phase'].str.startswith("OfflineRating")].copy().sort_values('line')
    df_offlines_1 = df[df['Phase'].str.startswith("OfflineRating")].copy().sort_values('line')
    if sum((df_offlines_1['Phase']=="OfflineRatingT0") & (df_offlines_1['TriggerName']=="fixaCue"))>0:
        print(df_offlines_1.shape)
        line_end_1 =  np.where((df_offlines_1['Phase']=="OfflineRatingT0") & (df_offlines_1['TriggerName']=="fixaCue"))[0][0]
        print(line_end_1)
        df_offlines_2 = df_offlines_1.iloc[0:line_end_1,:]
        line_end_2 =  np.where((df_offlines_2['Phase']=="OfflineRatingT0") & (df_offlines_2['TriggerName']=="instructions"))[0][-1]
        print(line_end_2)
        df_offlines_0 = df_offlines_2.iloc[0:line_end_2,:]
        print(df_offlines_0.shape)
        df_offlines_3s = df_offlines_1[df_offlines_1['Phase'].isin(["OfflineRatingT1","OfflineRatingT2","OfflineRatingT3"])]
        df_offlines = pd.concat([df_offlines_0, df_offlines_3s])
    else:
        df_offlines = df_offlines_1
    df_learning_checks = df[df['Phase'].str.startswith("LearningCheck")].copy().sort_values('line')
    df_demand_ratings = df[df['Phase'] == "DemandRating"].copy().sort_values('line')
    # combine into a dict
    df_physios_raw = {'Learning':df_learning, 'TrueChoice':df_true_choice, 'FalseChoice':df_false_choice, 'Offlines':df_offlines,
                     'LearningCheck':df_learning_checks, 'DemandRating':df_demand_ratings}
    return df_physios_raw

In [17]:
# add trial number
def add_trial_n(df):
    trial_n = []
    trial_count = 0            
    for i in range(df.shape[0]):
        if df['TriggerName'].iloc[i] == "trialStart":
            if df['TriggerName'].iloc[i+1] != "trialStart":
                trial_count += 1
        if df['TriggerName'].iloc[i] == "trialEnd":
            pass
        trial_n.append(trial_count)
    df_trialed = df.copy()
    df_trialed['Trial_N'] = trial_n
    return df_trialed

In [18]:
def ex_timeout(df,phase):
    rep = 0
    df_in = df.copy()
    if phase in ['TrueChoice','FalseChoice']:
        trig = "respChoiceTimeout"
    elif phase == 'Offlines':
        trig = "respOfflineRatingTimeout"
    else:
        trig = "hahaha"
    while sum(df['TriggerName']==trig):
        rep += 1
        trial_ex = set(df[df['TriggerName']==trig].Trial_N)
        trial_in = set(df.Trial_N) - trial_ex
        df_in = df[df['Trial_N'].isin(trial_in)].copy()
        # print('phase',phase)
        # print('trial_ex',trial_ex)
        # print('trial_in',trial_in)
        if rep == 3:
            break
    return df_in

In [19]:
def clean_phases(dfs, phase_names=['Learning','TrueChoice','FalseChoice','Offlines']):
    df_physios = {}
    for phase in phase_names:
        df_p = dfs[phase]
        df_trialed = add_trial_n(df_p)
        df_in = ex_timeout(df_trialed,phase)
        df_physios[phase] = df_in
    return df_physios

In [20]:
def ex_outliers_iqr(df, event, channel, gpb='label'):
    if channel in ['zyg','cor']:
        col = f'{event}_{channel}_IIRFilter_NotchFilter_femg_mean'
    elif channel == 'scr':
        col = f'{event}_{channel}_IIRFilter_DriverEstim_PhasicEstim_pha_DurationMean'
    iqrs = df.groupby(gpb)[col].quantile([0.05, 0.95]).unstack(level=1)
    low_outs = iqrs.loc[df[gpb],0.05] > df[col].values
    high_outs = iqrs.loc[df[gpb],0.95] < df[col].values
    ts = set(df['trialn'].values)
    t_outs = set(df.loc[(low_outs | high_outs).values]['trialn'].values)
    t_ins = ts - t_outs
    df_clean = df[df['trialn'].isin(t_ins)]
    return df_clean

In [21]:
def diff_values(df_c, trialn, event, channel):
    trials = range(1, trialn+1)
    diffs_dict = {'trials':trials}
    trials_in = set(df_c['trialn'].values)
    values = []
    if channel in ['zyg', 'cor']:
        col = f'{event}_{channel}_IIRFilter_NotchFilter_femg_mean'
    elif channel == 'scr':
        col = f'{event}_{channel}_IIRFilter_DriverEstim_PhasicEstim_pha_DurationMean'
    for i in trials:
        if i in trials_in:
            df = df_c.loc[df_c['trialn']==i,['label',col]]
            idx = list(df.columns).index(col)
            df.iloc[0, idx]
            fixa = df.iloc[0, idx]
            target = df.iloc[1, idx]
            diff = target - fixa
        else:
            diff = np.nan
        values.append(diff)
        zvalues = zscore(values, nan_policy='omit')
    diffs_dict[f'{event}_{channel}_values'] = values
    diffs_dict[f'{event}_{channel}_zvalues'] = zvalues
    df_diffs = pd.DataFrame(diffs_dict)
    return df_diffs

In [28]:
def signal_compute(df_p, samp_rate, event, trialn, channels=['zyg','cor','scr']):
    if event in ["cue_i_on"]:
        events = ['fixaCue', 'cueEasy', 'cueHard']
    elif event in ["cue_e_on","cue_e_true"]:
        events = ['fixaSeq', 'seqStart_N1', 'seqStart_N3']
    elif event == "cue_e_false":
        events = ['fixaSeq', 'seqStart_N2']
    elif event == "cue_i_off":
        events = ['fixaOfflineRating', 'cueEasy', 'cueHard']
    df = df_p[df_p['TriggerName'].isin(events)]
    label = signal.create_signal(df['Code'], sampling_freq = samp_rate, name = 'label')
    df_results = {}
    # filters
    for channel in channels:
        # print(channel)
        if channel in ['zyg', 'cor']:
            femg = signal.create_signal(df[channel], sampling_freq = samp_rate, name = f'{event}_{channel}')
            bandpass = filters.IIRFilter(fp=45, fs=450, btype='bandpass', ftype='butter')(femg)
            rectify = bandpass.copy()
            rectify[f'{event}_{channel}_IIRFilter'] = abs(bandpass[f'{event}_{channel}_IIRFilter'])
            notch = filters.NotchFilter(f=50)(rectify)
            norm = filters.Normalize(notch)
            df_filtered = notch
            # compute values
            segmenter = segmenters.LabelSegments(timeline=label, drop_mixed=False, drop_cut=False)
            results = segmenters.fmap(segmenter, preset_femg(), df_filtered)
        elif channel in ['scr']:
            eda = signal.create_signal(df[channel], sampling_freq = samp_rate, name = f'{event}_{channel}')
            eda /= 10
            bandpass = filters.IIRFilter(fp=0.8, fs=1.1, btype='bandpass', ftype='ellip')(eda)
            driver = DriverEstim()(bandpass)
            phasic = PhasicEstim(0.02)(driver)
            norm = filters.Normalize(phasic)
            df_filtered = phasic
            # compute values
            segmenter = segmenters.LabelSegments(timeline=label, drop_mixed=False, drop_cut=False)
            results = segmenters.fmap(segmenter, preset_phasic(delta=0.02), df_filtered)
        else:
            print('wrong channel!')
        df_res = results.to_dataframe()
        print('event',event,'channel',channel, 'shape',df_res.shape[0])
        tn = sorted(list(range(1, int(df_res.shape[0]/2)+1))*2)
        df_res['trialn'] = tn
        print(tn)
        df_res_clean = ex_outliers_iqr(df_res, event, channel)
        df_res_diff = diff_values(df_res_clean, trialn, event, channel)
        df_res_diff = df_res_diff.rename(columns={'trials':'TrialN'})
        df_results[channel] = df_res_diff
    
    return df_results

In [23]:
# def diff_values(dfs, trials, channels=['zyg','cor','scr']):
#     values = []
#     trials_in = set(df['trialn'].values)
#     for i in trials:
#         if i in trials_in:
#             df = df_res_clean.loc[df_res_clean['trialn']==i,['label',f'{channel}_IIRFilter_NotchFilter_femg_mean']]
#             idx = list(df.columns).index(f'{channel}_IIRFilter_NotchFilter_femg_mean')
#             df.iloc[0, idx]
#             fixa = df.iloc[0, idx]
#             target = df.iloc[1, idx]
#             diff = target - fixa
#         else:
#             diff = np.nan
#         values.append(diff)
#         zvalues = zscore(values, nan_policy='omit')
#     diffs = pd.DataFrame({'trials':trials, f'{channel}_values':values, f'{channel}_zvalues':zvalues})
#     return diffs

In [41]:
def compute_phases(dfs, trialns, phase_names=['Learning','TrueChoice','FalseChoice','Offlines'], samp_rate=1_000):
    df_results = {}
    for phase in phase_names:
        df = dfs[phase]
        trialn = trialns[phase]
        if phase == 'Learning':
            df_result = {}
            df_result_cue_i = signal_compute(df, samp_rate, "cue_i_on", trialn)
            df_result_cue_e = signal_compute(df, samp_rate, "cue_e_on", trialn)
            for channel in ['zyg','cor','scr']:
                df_result[channel] = df_result_cue_i[channel].merge(df_result_cue_e[channel], on='TrialN')
        elif phase == "TrueChoice":
            df_result = signal_compute(df, samp_rate, "cue_e_true", trialn)
        elif phase == "FalseChoice":
            df_result = signal_compute(df, samp_rate, "cue_e_false", trialn)
        elif phase == "Offlines":
            df_result = signal_compute(df, samp_rate, "cue_i_off", trialn)
        print(phase, df_result['cor'].columns)
        
        df_results[f'{phase}'] = df_result
    return df_results

In [42]:
# f_p = "/Users/yolo/Documents/Coding/fEMG/DACFE_45/042/Data/042.txt"
# df_phy_raw = load(f_p,triggers)
# df_physios_raw = phases(df_phy_raw)
df_physios = clean_phases(df_physios_raw)
df_physios_res = compute_phases(df_physios, TRIALN)
# df_physios_res_comb = {}
# for phase in ['Learning','TrueChoice','FalseChoice','Offlines']:
#     df_physios_res_comb[phase] = df_physios_res[phase]['zyg'][['trials']]
#     for channel in ['zyg','cor','scr']:
#         df_physios_res1 = df_physios_res[phase][channel]
#         df_physios_res_comb[phase] = df_physios_res_comb[phase].merge(df_physios_res1, on='trials')

Learning Index(['zyg', 'scr', 'cor', 'trg1', 'trg2', 'trg3', 'trg4', 'trg5', 'trg6',
       'trg7', 'trg8', 'na', 'line', 'binary', 'TriggerName', 'Code', 'Phase',
       'Trial_N'],
      dtype='object')
event cue_i_on channel zyg shape 100
[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50]
event cue_i_on channel cor shape 100
[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45,

/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt

event cue_i_on channel scr shape 100
[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50]
event cue_e_on channel zyg shape 100
[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50]
event cue_e_on channel cor shape 100
[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 

/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


event cue_e_on channel scr shape 100
[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50]
Learning Index(['TrialN', 'cue_i_on_cor_values', 'cue_i_on_cor_zvalues',
       'cue_e_on_cor_values', 'cue_e_on_cor_zvalues'],
      dtype='object')
TrueChoice Index(['zyg', 'scr', 'cor', 'trg1', 'trg2', 'trg3', 'trg4', 'trg5', 'trg6',
       'trg7', 'trg8', 'na', 'line', 'binary', 'TriggerName', 'Code', 'Phase',
       'Trial_N'],
      dtype='object')
event cue_e_true channel zyg shape 20
[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10]
event cue_e_true channel cor shape 20
[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10]
event cue_e_true chan

In [40]:
df_physios_res['FalseChoice']['TrialN']

KeyError: 'TrialN'

In [313]:
5df_physios['TrueChoice'].to_csv('/Users/yolo/Documents/Coding/fEMG/Preprocess/df_phy_clean_046.csv')

In [411]:
# f_p = "/Users/yolo/Documents/Coding/fEMG/DACFE_46/022/Data/022.txt"
# df_phy_raw = load(f_p,triggers)
df_phy_raw[df_phy_raw['line'].isin(range(5400000, 5401540))]

,zyg,scr,cor,trg1,trg2,trg3,trg4,trg5,trg6,trg7,trg8,na,line,binary,TriggerName,Code
1218098,-0.003967,0.329590,-0.005188,5,5,0,0,0,0,0,5,NaN,5400000,55000005,letterDiff,131
2288521,-0.006256,0.325012,-0.025940,5,0,0,5,5,5,0,0,NaN,5401490,50055500,trialEnd,57
2288522,-0.007019,0.329590,-0.018921,5,0,0,5,5,5,0,0,NaN,5401491,50055500,trialEnd,57
2288523,-0.006409,0.329590,-0.019989,5,0,0,5,5,5,0,0,NaN,5401492,50055500,trialEnd,57
2288524,-0.003815,0.326538,-0.011749,5,0,0,5,5,5,0,0,NaN,5401493,50055500,trialEnd,57
2288525,0.001831,0.326538,0.000153,5,0,0,5,5,5,0,0,NaN,5401494,50055500,trialEnd,57
2288526,0.001068,0.329590,0.003052,5,0,0,5,5,5,0,0,NaN,5401495,50055500,trialEnd,57
2288527,-0.001068,0.331116,0.003052,5,0,0,5,5,5,0,0,NaN,5401496,50055500,trialEnd,57
2288528,-0.000458,0.328064,0.011139,5,0,0,5,5,5,0,0,NaN,5401497,50055500,trialEnd,57
2288529,0.001831,0.328064,0.011902,5,0,0,5,5,5,0,0,NaN,5401498,50055500,trialEnd,57


In [407]:
# set(df_phy_raw['TriggerName'])
df_phy_raw[df_phy_raw['TriggerName']=="fixaProbChoice"]['line']

2559231    5401536
2559232    5401537
2559233    5401538
2559234    5401539
2559235    5401540
            ...   
2698456    6808442
2698457    6808443
2698458    6808444
2698459    6808445
2698460    6808446
Name: line, Length: 139230, dtype: int64

In [321]:
f_p = "/Users/yolo/Documents/Coding/fEMG/DACFE_46/045/Data/045.txt"
df_phy_raw = load(f_p,triggers)
df_physios_raw = phases(df_phy_raw)
df_physios = clean_phases(df_physios_raw)
df_physios_res = compute_phases(df_physios, TRIALN)

expStart
True
event cue_i_on channel zyg shape 100
event cue_i_on channel cor shape 100


/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


event cue_i_on channel scr shape 100
event cue_e_on channel zyg shape 100
event cue_e_on channel cor shape 100


/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt

event cue_e_on channel scr shape 100
Learning Index(['TrialN', 'cue_i_on_zyg_values', 'cue_i_on_zyg_zvalues',
       'cue_e_on_zyg_values', 'cue_e_on_zyg_zvalues'],
      dtype='object')
event cue_e_true channel zyg shape 20
event cue_e_true channel cor shape 20


/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


event cue_e_true channel scr shape 20
TrueChoice Index(['TrialN', 'cue_e_true_zyg_values', 'cue_e_true_zyg_zvalues'], dtype='object')
FalseChoice Index(['TrialN', 'cue_e_true_zyg_values', 'cue_e_true_zyg_zvalues'], dtype='object')
event cue_i_off channel zyg shape 16
event cue_i_off channel cor shape 16
event cue_i_off channel scr shape 16
Offlines Index(['TrialN', 'cue_i_off_zyg_values', 'cue_i_off_zyg_zvalues'], dtype='object')


In [322]:
df_physios_raw['Learning'].loc[df_physios['Learning']['TriggerName']=='trialStart']

,zyg,scr,cor,trg1,trg2,trg3,trg4,trg5,trg6,trg7,trg8,na,line,binary,TriggerName,Code,Phase
1367266,-0.003052,1.193240,-0.000305,0,0,0,5,5,5,0,0,NaN,919099,00055500,trialStart,56,Learning
1367267,-0.002747,1.196290,-0.006714,0,0,0,5,5,5,0,0,NaN,919100,00055500,trialStart,56,Learning
1367268,-0.001526,1.196290,-0.006866,0,0,0,5,5,5,0,0,NaN,919101,00055500,trialStart,56,Learning
1367269,-0.010376,1.196290,-0.005646,0,0,0,5,5,5,0,0,NaN,919102,00055500,trialStart,56,Learning
1367270,-0.000763,1.196290,-0.001221,0,0,0,5,5,5,0,0,NaN,919103,00055500,trialStart,56,Learning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367766,-0.002441,0.645447,-0.000916,0,0,0,5,5,5,0,0,NaN,2192205,00055500,trialStart,56,Learning
1367767,-0.014954,0.645447,-0.005798,0,0,0,5,5,5,0,0,NaN,2192206,00055500,trialStart,56,Learning
1367768,-0.015564,0.646973,0.004883,0,0,0,5,5,5,0,0,NaN,2192207,00055500,trialStart,56,Learning
1367769,-0.006409,0.650025,0.001526,0,0,0,5,5,5,0,0,NaN,2192208,00055500,trialStart,56,Learning


In [171]:
df_p = df_physios['Offlines']
events = ['fixaOfflineRating', 'cueEasy', 'cueHard']
samp_rate = 1_000
# df_p

In [168]:
set(df_p['Trial_N'])

{0, 1, 2, 3, 4, 5, 6, 7, 8}

In [129]:
np.where((df_phy_raw['Phase']=="OfflineRatingT0") & (df_phy_raw['TriggerName']=="instructions"))[0][-1]

926991

In [98]:
# df_p.to_csv('/Users/yolo/Documents/Coding/fEMG/Preprocess/df_offlines_check_009.csv')

In [229]:
# df = df_p[df_p['TriggerName'].isin(events)]
# label = signal.create_signal(df['Code'], sampling_freq = samp_rate, name = 'label')
# eda = signal.create_signal(df_physios['Learning']['scr'], sampling_freq = 1_000, name = 'scr')
# eda /= 10
# bandpass = filters.IIRFilter(fp=0.8, fs=1.1, btype='bandpass', ftype='ellip')(eda)
# driver = DriverEstim()(bandpass)
# phasic = PhasicEstim(0.02)(driver)
# # norm = filters.Normalize(phasic)
# df_filtered = phasic
# # # compute values
# segmenter = segmenters.LabelSegments(timeline=label, drop_mixed=False, drop_cut=False)
# results = segmenters.fmap(segmenter, preset_phasic(delta=0.02), df_filtered)

In [230]:
# results.to_dataframe()

In [58]:
f_p = "/Users/yolo/Documents/Coding/fEMG/DACFE_6/007/Data/007.txt"
df_phy_raw = load(f_p,triggers)
df_physios_raw = phases(df_phy_raw)
df_physios = clean_phases(df_physios_raw)
df_p = df_physios['Offlines']
events = ['fixaCue', 'cueEasy', 'cueHard']
samp_rate = 1_000
df = df_p[df_p['TriggerName'].isin(events)]
# label = signal.create_signal(df['Code'], sampling_freq = samp_rate, name = 'label')
# eda = signal.create_signal(df_physios['Learning']['scr'], sampling_freq = 1_000, name = 'scr')
# eda /= 10
# bandpass = filters.IIRFilter(fp=0.8, fs=1.1, btype='bandpass', ftype='ellip')(eda)
# driver = DriverEstim()(bandpass)
# phasic = PhasicEstim(0.02)(driver)
# # norm = filters.Normalize(phasic)
# df_filtered = phasic
# # # compute values
# segmenter = segmenters.LabelSegments(timeline=label, drop_mixed=False, drop_cut=False)
# results = segmenters.fmap(segmenter, preset_phasic(delta=0.02), df_filtered)
# results.to_dataframe()

In [71]:
df_phy_raw.to_csv('/Users/yolo/Documents/Coding/fEMG/Preprocess/df_raw_007.csv')

In [67]:
df_p.to_csv('/Users/yolo/Documents/Coding/fEMG/Preprocess/df_offlines_check_007.csv')

In [108]:
# df_p = df_physios['Learning']
# events = ['fixaSeq', 'seqStart_N1', 'seqStart_N3']
# samp_rate = 1_000
# df = df_p[df_p['TriggerName'].isin(events)]
# label = signal.create_signal(df['Code'], sampling_freq = samp_rate, name = 'label')
# eda = signal.create_signal(df_physios['Learning']['scr'], sampling_freq = 1_000, name = 'scr')
# eda /= 10
# bandpass = filters.IIRFilter(fp=0.8, fs=1.1, btype='bandpass', ftype='ellip')(eda)
# driver = DriverEstim()(bandpass
# phasic = PhasicEstim(0.02)(driver)
# # norm = filters.Normalize(phasic)
# df_filtered = phasic
# # # compute values
# segmenter = segmenters.LabelSegments(timeline=label, drop_mixed=False, drop_cut=False)
# results = segmenters.fmap(segmenter, preset_phasic(delta=0.02), df_filtered)

In [15]:
# df_physios['Learning'][df_physios['Learning']['TriggerName'].isin(['fixaSeq', 'seqStart_N1', 'seqStart_N3'])].groupby('Code').mean()